In [2]:
import numpy as np
import numpy.random as nrand
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools
from itertools import combinations
import copy

In [97]:
BASE_list = ['A','C','G','U']

In [149]:
#df_all = pd.read_csv('Supplemental_File_S3.csv',skiprows=5,index_col=False,names=['seq','P1','P2','P3','mean'])
df_all = pd.read_csv('SD_seq_arti_data.csv',index_col=False)
seqs_raw = df_all.seq.to_numpy()
seqs = np.array(list(map(list,seqs_raw)))

#df_all[df_all.iloc[:,1:4].isna().sum(axis=1) == 0].to_csv('SD_seq_arti_data.csv',index=None)

197890

In [151]:
seqs_list = [''.join(seq) for seq in seqs]
seqidx_dict = {seqs_list[i]:i for i in range(len(df_all))}

In [159]:
#Generate neighbor_list for N_max calculation

neighbor_list = []
for N,seq in enumerate(seqs):
    print(N,end='\r')
    neighbor_list_sub = []
    for i in range(9):
        seq_tmp = np.copy(seq)
        for BASE in BASE_list:
            if BASE == seq[i]:
                continue
            else: seq_tmp[i] = BASE
            if ''.join(seq_tmp) in seqidx_dict:
                neighbor_list_sub.append(seqidx_dict[''.join(seq_tmp)])
    neighbor_list.append(neighbor_list_sub)

neighbor_list = np.array(neighbor_list)
# with open('../../index_file/SD_seq_neighbor_list.pkl','wb') as f:
#     pickle.dump(neighbor_list,f)

In [163]:
# # Generate epi_square_list for epi calculation

epi_square_list = []
for idx_00, seq_00 in enumerate(seqs): 
    print(idx_00,end='\r')
    for pos1,pos2 in itertools.combinations(range(9),2):
        seq_10 = np.copy(seq_00)
        seq_01 = np.copy(seq_00)
        seq_11 = np.copy(seq_00)
        for mut1 in BASE_list:
            if mut1 == seq_00[pos1]:
                continue
            seq_10[pos1] = mut1
            for mut2 in BASE_list:
                if mut2 == seq_00[pos2]:
                    continue
                seq_01[pos2] = mut2
                seq_11[pos1] = mut1; seq_11[pos2] = mut2
                
                if ''.join(seq_10) not in seqidx_dict or \
                   ''.join(seq_01) not in seqidx_dict or \
                   ''.join(seq_11) not in seqidx_dict:
                    continue
                idx_10 = seqidx_dict[''.join(seq_10)]
                idx_01 = seqidx_dict[''.join(seq_01)]
                idx_11 = seqidx_dict[''.join(seq_11)]
                if idx_00 >= min([idx_10, idx_01, idx_11]):
                    continue
                elif idx_10 < idx_01:
                    epi_square_list.append([idx_00,idx_10,idx_01,idx_11])
                elif idx_10 > idx_01:
                    epi_square_list.append([idx_00,idx_01,idx_10,idx_11])

epi_square_list = np.array(epi_square_list)
#with open('../../index_file/SD_seq_epi_square_list.pkl','wb') as f:
#    pickle.dump(epi_square_list,f)

In [197]:
nrand.seed(48109)
idx_0_list = nrand.choice(len(seqs),size=450, replace=False)

In [198]:
# Generate pathway_list for open_ratio calculation

pathway_list = []
for N,idx_0 in enumerate(idx_0_list):
    print(N,end='\r')
    seq_0 = seqs[idx_0]
    idx_4_list = np.where((seqs != seq_0).sum(axis=1) == 4)[0]
    idx_4_list_filtered = idx_4_list[idx_4_list>idx_0]
    for idx_4 in idx_4_list_filtered:
        seq_4 = seqs[idx_4]
        poss = np.where(seq_4 != seq_0)[0]
        muts = seq_4[seq_4 != seq_0]
        
        for order in itertools.permutations(range(4),3):
            pathway_sub_list = [idx_0]
            seq_tmp = np.copy(seq_0)
            for step in range(3):
                pos = poss[order[step]]; mut = muts[order[step]]
                seq_tmp[pos] = mut
                seq_tmp_string = ''.join(seq_tmp)
                if seq_tmp_string in seqidx_dict:
                    pathway_sub_list.append(seqidx_dict[seq_tmp_string])
                else:
                    break
                if step == 2:
                    pathway_sub_list.append(idx_4)
                    pathway_list.append(pathway_sub_list)

#pathway_list = np.array(pathway_list[0:25000000])
# with open('../../index_file/SD_seq_pathway_list_4steps_25000000.pkl','wb') as f:
#     pickle.dump(pathway_list,f)